In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from IPython.display import display

In [0]:
# !unzip -q ml-100k.zip
# !rm ml-100k.zip

In [0]:
#Reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols, encoding='latin-1')

#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols, encoding='latin-1')

#Reading items file:
i_cols = ['movie id', 'movie title' ,'release date','video release date', \
          'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', \
          'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', \
          'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1')

In [4]:
display(users); display(ratings); display(items);

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


# Pre-process of data

In [5]:
ratings

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [6]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
user_id           100000 non-null int64
movie_id          100000 non-null int64
rating            100000 non-null int64
unix_timestamp    100000 non-null int64
dtypes: int64(4)
memory usage: 3.1 MB


In [0]:
ratings['user_id'] = ratings['user_id'].astype('object')
ratings['movie_id'] = ratings['movie_id'].astype('object')
ratings['date'] = pd.to_datetime(ratings['unix_timestamp'], unit='s')

ratings.drop(columns=['unix_timestamp'], inplace=True)

In [8]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
user_id     100000 non-null object
movie_id    100000 non-null object
rating      100000 non-null int64
date        100000 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 3.1+ MB


In [9]:
ratings['date'].describe()

count                  100000
unique                  49282
top       1998-03-27 21:20:06
freq                       12
first     1997-09-20 03:05:10
last      1998-04-22 23:10:38
Name: date, dtype: object

In [0]:
# literally drop this column
ratings.drop(columns=['date'], inplace=True)

### Movies Dataframe

In [11]:
items

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 24 columns):
movie id              1682 non-null int64
movie title           1682 non-null object
release date          1681 non-null object
video release date    0 non-null float64
IMDb URL              1679 non-null object
unknown               1682 non-null int64
Action                1682 non-null int64
Adventure             1682 non-null int64
Animation             1682 non-null int64
Children's            1682 non-null int64
Comedy                1682 non-null int64
Crime                 1682 non-null int64
Documentary           1682 non-null int64
Drama                 1682 non-null int64
Fantasy               1682 non-null int64
Film-Noir             1682 non-null int64
Horror                1682 non-null int64
Musical               1682 non-null int64
Mystery               1682 non-null int64
Romance               1682 non-null int64
Sci-Fi                1682 non-null int64
Thriller 

In [13]:
items.drop(columns=['release date', 'video release date', 'IMDb URL'], inplace=True)
items

,movie id,movie title,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
items.iloc[:, 2:].sum()

unknown          2
Action         251
Adventure      135
Animation       42
Children's     122
Comedy         505
Crime          109
Documentary     50
Drama          725
Fantasy         22
Film-Noir       24
Horror          92
Musical         56
Mystery         61
Romance        247
Sci-Fi         101
Thriller       251
War             71
Western         27
dtype: int64

In [0]:
# Drop 'unknown' too
items.drop(columns=['unknown'], inplace=True)

In [16]:
items.head(3)

,movie id,movie title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


### DF to Dict

In [0]:
def df2dict(df):
    id2movie, movie2id, id2feature = {}, {}, {}
    for idx in df.index:
        id2movie[df.iloc[idx, 0]] = df.iloc[idx, 1]
        movie2id[df.iloc[idx, 1]] = df.iloc[idx, 0]
        id2feature[df.iloc[idx, 0]] = np.array(df.iloc[idx, 2:].values, dtype=np.int64)

    return id2movie, movie2id, id2feature

id2movie, movie2id, id2feature = df2dict(items)

In [18]:
print(id2movie[1], movie2id['Toy Story (1995)'], id2feature[1], sep='\n')

Toy Story (1995)
1
[0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]


# Algorithms

## Cosine Similarity

In [0]:
def cosine_similarity(u, v):
    
    dot = np.dot(u, v)

    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)
    mul = norm_u * norm_v

    cosine_similarity = (dot / mul) if mul != 0 else 0
    
    return cosine_similarity

In [0]:
def recommend_movies(movie_name, top=10, max_cosine_sim=0.6):
    
    current_feature = id2feature[movie2id[movie_name]]
    
    movie_ids = id2feature.keys()
    best_movies = []                    
    
    # loop over the whole word vector set
    for idx in movie_ids:        
        temp_feature = id2feature[idx]
        cosine_sim = cosine_similarity(current_feature, temp_feature)

        if cosine_sim >= max_cosine_sim:
            best_movies.append((id2movie[idx], cosine_sim))
    best_movies = sorted(best_movies, key=lambda x: x[1], reverse=True)[:top]
    # best_movies = list(map(lambda x: x[0], best_movies))

    return best_movies

In [21]:
recommend_movies('Toy Story (1995)')

[('Toy Story (1995)', 1.0000000000000002),
 ('Aladdin and the King of Thieves (1996)', 1.0000000000000002),
 ('Aladdin (1992)', 0.8660254037844387),
 ('Goofy Movie, A (1995)', 0.8660254037844387),
 ('Santa Clause, The (1994)', 0.8164965809277259),
 ('Home Alone (1990)', 0.8164965809277259),
 ('Aristocats, The (1970)', 0.8164965809277259),
 ('D3: The Mighty Ducks (1996)', 0.8164965809277259),
 ('Love Bug, The (1969)', 0.8164965809277259),
 ('Wrong Trousers, The (1993)', 0.8164965809277259)]

## Euclidean Distance

In [0]:
def euclidean_distance(u, v):    
    return np.linalg.norm(u - v)

def recommend_movies2(movie_name, top=10, min_e_d=1.0):
    current_feature = id2feature[movie2id[movie_name]]
    
    movie_ids = id2feature.keys()
    best_movies = [] 

    # loop over the whole word vector set
    for idx in movie_ids:        
        temp_feature = id2feature[idx]
        e_d = euclidean_distance(current_feature, temp_feature)

        if e_d <= min_e_d:
            best_movies.append((id2movie[idx], e_d))
    best_movies = sorted(best_movies, key=lambda x: x[1])[:top]
    # best_movies = list(map(lambda x: x[0], best_movies))

    return best_movies

In [23]:
recommend_movies2('Toy Story (1995)', min_e_d=1.0)

[('Toy Story (1995)', 0.0),
 ('Aladdin and the King of Thieves (1996)', 0.0),
 ('Santa Clause, The (1994)', 1.0),
 ('Home Alone (1990)', 1.0),
 ('Aladdin (1992)', 1.0),
 ('Aristocats, The (1970)', 1.0),
 ('D3: The Mighty Ducks (1996)', 1.0),
 ('Love Bug, The (1969)', 1.0),
 ('Wrong Trousers, The (1993)', 1.0),
 ('Grand Day Out, A (1992)', 1.0)]

## [Analogy] Cosine Similarity works better than Euclidean Distance

In [24]:
# See some data
display(items[items['movie title'] == 'Toy Story (1995)'])
display(items[items['movie title'] == 'Aladdin and the King of Thieves (1996)'])
display(items[items['movie title'] == 'Aladdin (1992)'])

,movie id,movie title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0


,movie id,movie title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
421,422,Aladdin and the King of Thieves (1996),0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0


,movie id,movie title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
94,95,Aladdin (1992),0,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0
